## Peri-Event Time Histogram (PETH) with Pynapple

A **PETH** aligns neural activity to behavioral events and averages across trials. [Pynapple](https://pynapple.org) makes this simple:

- `nap.compute_perievent()` - align spikes to events
- `.to_tsd()` - flatten for raster plots  
- `.count(bin_size)` - bin spikes for firing rate

In [2]:
import pynapple as nap
import numpy as np
import matplotlib.pyplot as plt
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient
import remfile
import h5py

# Stream NWB file from DANDI
dandiset_id = "001636"
session_id = "V++v4529++PostMPTP++Depth20600um++20000127"

client = DandiAPIClient()
dandiset = client.get_dandiset(dandiset_id)
assets = list(dandiset.get_assets())
asset = next(a for a in assets if session_id in a.path)
print(f"Streaming: {asset.path}")

# Open remote file
url = asset.get_content_url(follow_redirects=1, strip_query=True)
file = remfile.File(url)
h5_file = h5py.File(file, "r")
io = NWBHDF5IO(file=h5_file, load_namespaces=True)
nwbfile = io.read()
data = nap.NWBFile(nwbfile)
print(f"Session: {nwbfile.session_id}")

Streaming: sub-V/sub-V_ses-V++v4529++PostMPTP++Depth20600um++20000127_behavior+ecephys.nwb
Session: V++v4529++PostMPTP++Depth20600um++20000127


In [6]:
data

V++v4529++PostMPTP++Depth20600um++20000127
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┑
│ Keys                               │ Type        │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┥
│ units                              │ TsGroup     │
│ trials                             │ IntervalSet │
│ AntidromicSweepsIntervals          │ IntervalSet │
│ LFP                                │ TsdFrame    │
│ ElbowVelocity                      │ Tsd         │
│ AntidromicStimulationUnit1Striatum │ Tsd         │
│ AntidromicResponseUnit1Striatum    │ TsdFrame    │
│ ElbowTorque                        │ Tsd         │
│ ElbowAngle                         │ TsdFrame    │
│ EMGTricepsLongus2                  │ Tsd         │
│ EMGTricepsLongus1                  │ Tsd         │
│ EMGTrapezius                       │ Tsd         │
│ EMGDeltoidPosterior                │ Tsd         │
│ EMGBrachioradialis                 │ Tsd         │
┕━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━┙

In [ ]:
# Extract data
trials = data["trials"]
spikes = data["units"][0]
velocity = data["ElbowVelocity"]

# Split by movement type
flex_trials = trials[trials["movement_type"] == "flexion"]
ext_trials = trials[trials["movement_type"] == "extension"]
flex_onset = nap.Ts(flex_trials["derived_movement_onset_time"].values)
ext_onset = nap.Ts(ext_trials["derived_movement_onset_time"].values)

# Compute PETH - align spikes and velocity to movement onset
window = (-0.5, 1.0)  # 500ms before to 1000ms after movement onset
bin_size = 0.01       # 10ms bins for firing rate

# 1. Align spikes to movement onset for each condition
#    Returns TsGroup where each element is one trial's spikes (time relative to onset)
peth_flex = nap.compute_perievent(spikes.restrict(flex_trials), flex_onset, minmax=window)
peth_ext = nap.compute_perievent(spikes.restrict(ext_trials), ext_onset, minmax=window)

# 2. Align velocity to movement onset
#    Returns TsdFrame with shape (n_timepoints, n_trials)
vel_flex = nap.compute_perievent_continuous(velocity.restrict(flex_trials), flex_onset, minmax=window)
vel_ext = nap.compute_perievent_continuous(velocity.restrict(ext_trials), ext_onset, minmax=window)

# 3. Flatten spikes for raster plot using .to_tsd()
#    Values = trial index, times = spike times relative to onset
raster_flex = peth_flex.to_tsd()
raster_ext = peth_ext.to_tsd()

# 4. Compute smoothed firing rate using .count() and .smooth()
#    .count(bin_size) bins spikes, .smooth(std) applies Gaussian kernel
#    Divide by bin_size to convert from spike count to Hz
fr_flex = peth_flex.count(bin_size).smooth(std=0.025)  # 25ms Gaussian smoothing
fr_ext = peth_ext.count(bin_size).smooth(std=0.025)

print(f"Flexion: {len(flex_trials)} trials, Extension: {len(ext_trials)} trials")
print(f"Velocity shape: {vel_flex.shape} (timepoints x trials)")
print(f"Firing rate shape: {fr_flex.shape} (timepoints x trials)")

In [ ]:
# Plot PETH
BLUE, RED = "#2563EB", "#DC2626"

fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
window_ms = (window[0]*1000, window[1]*1000)

# Row 1: Velocity
t_ms = vel_flex.times() * 1000
axes[0].plot(t_ms, np.nanmean(vel_flex.values, 1), lw=3, color=BLUE, label=f"Flexion (n={len(flex_trials)})")
axes[0].plot(t_ms, np.nanmean(vel_ext.values, 1), lw=3, color=RED, label=f"Extension (n={len(ext_trials)})")
axes[0].axvline(0, color='k', ls='--', lw=2)
axes[0].axhline(0, color='gray', alpha=0.3)
axes[0].set_ylabel("Velocity (deg/s)", fontsize=14, fontweight='bold')
axes[0].set_title("Elbow Velocity", fontsize=16, fontweight='bold')
axes[0].legend(fontsize=14, frameon=False)

# Row 2: Raster
axes[1].plot(raster_flex.times()*1000, raster_flex.values, '|', ms=8, mew=1.5, color=BLUE)
axes[1].plot(raster_ext.times()*1000, raster_ext.values + len(peth_flex), '|', ms=8, mew=1.5, color=RED)
axes[1].axhline(len(peth_flex)-0.5, color='gray', lw=2)
axes[1].axvline(0, color='k', ls='--', lw=2)
axes[1].set_ylabel("Trial", fontsize=14, fontweight='bold')
axes[1].set_title("Spike Raster", fontsize=16, fontweight='bold')

# Row 3: Firing Rate
t_fr = fr_flex.times() * 1000
axes[2].plot(t_fr, np.nanmean(fr_flex.values, 1)/bin_size, lw=3, color=BLUE)
axes[2].plot(t_fr, np.nanmean(fr_ext.values, 1)/bin_size, lw=3, color=RED)
axes[2].axvline(0, color='k', ls='--', lw=2)
axes[2].set_xlabel("Time from movement onset (ms)", fontsize=14, fontweight='bold')
axes[2].set_ylabel("Firing Rate (Hz)", fontsize=14, fontweight='bold')
axes[2].set_title("PETH", fontsize=16, fontweight='bold')
axes[2].set_ylim(bottom=0)

for ax in axes:
    ax.set_xlim(*window_ms)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(labelsize=12)

plt.tight_layout()
plt.show()

---

## How It Works

### The Key Functions

**1. `nap.compute_perievent(spikes, events, minmax)`** - Aligns spikes to events, returns `TsGroup` with one element per trial

**2. `.to_tsd()`** - Flattens TsGroup for raster plots (values = trial index)

**3. `.count(bin_size)`** - Bins spikes into time bins

**4. `.smooth(std)`** - Gaussian smoothing (e.g., `std=0.025` for 25ms); divide by bin_size for firing rate

### Minimal Example

```python
# Align spikes to events
peth = nap.compute_perievent(spikes, events, minmax=(-0.5, 1.0))

# Raster plot
plt.plot(peth.to_tsd(), '|')

# Firing rate (smoothed)
plt.plot(np.mean(peth.count(0.01).smooth(std=0.025), 1) / 0.01)
```